In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil
import random
from skimage.filters import gabor
import skimage.exposure
import SimpleITK as sitk
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
import scipy.ndimage  # Per il resize volumetrico



# ANALISI DEI METADATI DEI FILE NII
## Prima di effettuare delle operazioni sui file nii, verifica se la distanza tra le slide sia inferiore al millimetro e verifica se lo spazio dei voxel sia lo stesso (Es: 0.5,0.5,0.5)

In [0]:
# ANALISI METADATI DEI FILE NII
import nibabel as nib

# Percorso del file NIfTI
nii_path = "D:\StefanoTesiMagistrale\DatasetStefano\MSSEG-Training\Training\TrainingFlairNII\TrainANii\center_01_sub_01_FLAIR_preprocessed.nii"  # <-- modifica con il tuo percorso

# Carica immagine e header
img = nib.load(nii_path)
header = img.header
affine = img.affine

# Estrai informazioni
shape = img.shape
voxel_spacing = header.get_zooms()  # es. (1.0, 1.0, 1.0)
slice_spacing = voxel_spacing[2] if len(voxel_spacing) >= 3 else "Non disponibile"
data_type = header.get_data_dtype()

# Stampa
print(f"🧠 File: {nii_path}")
print(f"Forma volume (x, y, z): {shape}")
print(f"Datatype: {data_type}")
print(f"Voxel spacing (x, y, z): {voxel_spacing}")
print(f"➡️  Distanza tra le slice (asse Z): {slice_spacing} mm")
print(f"\nMatrice affine:\n{affine}")


In [0]:
#PER IMMAGINE SINGOLA

# Carica volume
vol = nib.load(r"D:\StefanoTesiMagistrale\DatasetStefano\DatasetNoSkullStripping\Dataset\ANAT\FileNII\TrainANii\sub-02_ses-Y3CMH_run-1_T1w_defaced.nii".get_fdata()

# Seleziona una singola slice
slice_index = 150
slice_data = vol[:, :, slice_index]

# Clipping tra 1° e 99° percentile
v_min, v_max = np.percentile(slice_data, (1, 99))
clipped = np.clip(slice_data, v_min, v_max)

# Normalizzazione tra 0 e 1
normalized = (clipped - v_min) / (v_max - v_min)

# Visualizza la slice
plt.imshow(normalized, cmap='gray', interpolation='bilinear')
plt.axis('off')  # Nasconde gli assi
plt.show()

In [0]:
# Per spacchettare in immagini png un file nii di tipo FLAIR

# Impostazioni e percorsi
target_spacing = 0.5  # Voxel isotropi desiderati: 0.5 x 0.5 x 0.5 mm

# Specifica il singolo file da processare
input_file = r"D:\StefanoTesiMagistrale\DatasetStefano\MSSEG-Training\Training\TrainingFlairNII\TrainANii\TrainA.nii"
output_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\MSSEG-Training\Training\TrainingFlair\TrainA"

# Crea la cartella di output se non esiste
os.makedirs(output_folder, exist_ok=True)

# Carica il file NIfTI
print(f"Processing {os.path.basename(input_file)}...")
img = nib.load(input_file)
vol = img.get_fdata()
print(f"Forma volume originale: {vol.shape}")

# Ottieni voxel spacing originale
voxel_spacing = img.header.get_zooms()
print("Voxel spacing originale:", voxel_spacing)

# Calcola fattori di zoom
zoom_factors = tuple(vs / target_spacing for vs in voxel_spacing)
print(f"Zoom factors: {zoom_factors}")

# Ricampiona il volume
vol_isotropic = scipy.ndimage.zoom(vol, zoom_factors, order=3)
print(f"Forma volume isotropico: {vol_isotropic.shape}")

# Elabora e salva ogni slice
for i in range(vol_isotropic.shape[2]):
    slice_data = vol_isotropic[:, :, i]

    v_min, v_max = np.percentile(slice_data, (5, 95))
    if np.abs(v_max - v_min) < 1e-6:
        print(f"Slice {i} ha range troppo piccolo, ignorata.")
        continue

    clipped = np.clip(slice_data, v_min, v_max)
    normalized = (clipped - v_min) / (v_max - v_min)

    m = normalized.mean()
    if m < 0.06 or m > 0.93:
        print(f"Slice {i} ha media molto estrema ({m:.2f}), ignorata.")
        continue

    plt.figure(figsize=(30, 30), dpi=150)
    plt.imshow(normalized, cmap='gray', interpolation='bilinear', aspect='equal')
    plt.axis('off')

    base_name = os.path.splitext(os.path.basename(input_file))[0]
    output_path = os.path.join(output_folder, f"{base_name}_slice{i}.png")
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

print("✅ Completato!")


In [0]:
# PER SPACCHETTARE TUTTI I FILE NII T1
# Percorsi dove si trova il dataset dei file .nii
input_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\MSSEG-Training\Training\TrainingFlairNII\TrainANii"
output_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\MSSEG-Training\Training\TrainingFlair\TrainB"


# Scorri tutti i file .nii o .nii.gz
for filename in os.listdir(input_folder):
    if filename.endswith(".nii"):
        filepath = os.path.join(input_folder, filename)
        print(f"Processing {filename}...")

        # Carica il volume
        vol = nib.load(filepath).get_fdata()
        print(vol.shape)
        # Per ogni slice selezionata
        for i in range(178, 450):
            if i >= vol.shape[2]:  # Evita slice out-of-bounds
                continue

            slice_data = vol[:, :, i]

            # Clipping + normalizzazione
            v_min, v_max = np.percentile(slice_data, (1, 99))
            clipped = np.clip(slice_data, v_min, v_max)
            normalized = (clipped - v_min) / (v_max - v_min)

            # Salva la slice
            plt.figure(figsize=(30, 30), dpi=150)
            plt.imshow(normalized, cmap='gray', interpolation='bilinear')
            plt.axis('off')

            # Nome immagine output
            base_name = os.path.splitext(filename)[0].replace('.nii', '')
            output_path = os.path.join(output_folder, f"{base_name}_slice{i}.png")

            plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
            plt.close()  # Libera memoria figura

print("✅ Completato!")


# TARGET SPACING DEL VOLUME in (0.5,0.5,0.5)
# Questo vale anche nel caso in cui si vuole avere un target_space di 1,2,ecc...

Questo codice serve nel caso in cui, visualizzando lo spazio di un voxel, non abbiamo la stessa misura su tutti e 3 gli assi (x,y,z) (Ad esempio: (1,0.5,0.5)). In questo caso si utilizza questo codice che crea degli zooming factor dividendo il corrente voxel spacing originale per lo spacing target.
Facendo la *divisione* tra gli space voxels e gli zooming factor otterrai lo space target.

## Esempio
Supponi di avere uno spazio di voxel pari a *(1,0.5,0.5)* e supponi che hai un target space di *(0.5)* . Allora gli zooming factors saranno pari a:
- 1/0.5 = 0.5
- 0.5/0.5 = 1
- 0.5/0.1 = 1



In [ ]:

import numpy as np
import nibabel as nib
import scipy.ndimage
import os
import matplotlib.pyplot as plt

# Impostazioni e percorsi
target_spacing = 0.5   # Obiettivo : voxel di 0.5x0.5x0.5 mm
input_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\MSSEG-Training\Training\TrainingFlairNII\TrainANii"
output_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\MSSEG-Training\Training\TrainingFlair\TrainA"

# Processa tutti i file .nii nella cartella di input
for filename in os.listdir(input_folder):
    if filename.endswith(".nii"):
        filepath = os.path.join(input_folder, filename)
        print(f"Processing {filename}...")
        img = nib.load(filepath)
        vol = img.get_fdata()
        print(f"Forma volume originale: {vol.shape}")

        # Estrai lo spacing originale dai metadati (es. (1.1, 0.5, 0.5))
        voxel_spacing = img.header.get_zooms()
        print("Voxel spacing originale:", voxel_spacing)

        # Calcola i fattori di zoom per ottenere voxel isotropi di target_spacing mm
        zoom_x = voxel_spacing[0] / target_spacing  # es., 1.1/0.5 = 2.2
        zoom_y = voxel_spacing[1] / target_spacing  # es., 0.5/0.5 = 1.0
        zoom_z = voxel_spacing[2] / target_spacing  # es., 0.5/0.5 = 1.0
        zoom_factors = (zoom_x, zoom_y, zoom_z)
        print(f"Zoom factors: {zoom_factors}")

        # Ricampiona l'intero volume per ottenere voxel isotropi (0.5, 0.5, 0.5)
        vol_isotropic = scipy.ndimage.zoom(vol, zoom_factors, order=3)
        print(f"Forma volume isotropico: {vol_isotropic.shape}")

        # Salva ogni slice (lungo l'asse Z) del volume isotropico, ignorando quelle con dinamiche troppo estreme
        for i in range(vol_isotropic.shape[2]):
            slice_data = vol_isotropic[:, :, i]
            # Utilizza percentili (5,95) per stabilire gli estremi per il clipping
            v_min, v_max = np.percentile(slice_data, (5, 95))
            if np.abs(v_max - v_min) < 1e-6:
                print(f"Slice {i} di {filename} ha range di intensità troppo piccolo, ignorata.")
                continue
            clipped = np.clip(slice_data, v_min, v_max)
            normalized = (clipped - v_min) / (v_max - v_min)
            
            # Se la slice risulta estremamente uniforme, ignorala
            m = normalized.mean()
            if m < 0.06 or m > 0.93:
                print(f"Slice {i} di {filename} ha media molto estrema ({m:.2f}), ignorata.")
                continue
            
            plt.figure(figsize=(30, 30), dpi=150)
            plt.imshow(normalized, cmap='gray', interpolation='bilinear', aspect='equal')
            plt.axis('off')
            base_name = os.path.splitext(filename)[0]
            output_path = os.path.join(output_folder, f"{base_name}_slice{i}.png")
            plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
            plt.close()
print("✅ Completato!")


Processing center_01_sub_01_FLAIR_preprocessed.nii...
Forma volume originale: (144, 512, 512)
Voxel spacing originale: (1.0999999, 0.5, 0.5)
Zoom factors: (2.1999998092651367, 1.0, 1.0)
Forma volume isotropico: (317, 512, 512)
Slice 0 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo piccolo, ignorata.
Slice 1 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo piccolo, ignorata.
Slice 2 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo piccolo, ignorata.
Slice 3 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo piccolo, ignorata.
Slice 4 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo piccolo, ignorata.
Slice 5 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo piccolo, ignorata.
Slice 6 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo piccolo, ignorata.
Slice 7 di center_01_sub_01_FLAIR_preprocessed.nii ha range di intensità troppo 

# Conto del numero di file nelle cartelle

In [0]:
folders = {
    "TrainA": r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TrainA",
    "TrainB": r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TrainB",
    "TestA":  r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TestA",
    "TestB":  r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TestB"
}

for name, path in folders.items():
    count = len([f for f in os.listdir(path) if f.lower().endswith('.png')])
    print(f"{name}: {count} immagini PNG")

# Spostare il 20% dei file da una cartella all'altra
utile se si deve splittare il dataset in train e test

In [ ]:


# Cartelle originali
trainA_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TrainA"
trainB_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TrainB"

# Cartelle di destinazione
testA_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TestA"
testB_folder = r"D:\StefanoTesiMagistrale\DatasetStefano\Dataset\ANAT\TestB"

def move_percentage_of_files(src_folder, dst_folder, percentage):
    files = [f for f in os.listdir(src_folder) if f.lower().endswith('.png')]
    random.shuffle(files)
    n_to_move = int(len(files) * percentage)

    for file in files[:n_to_move]:
        src_path = os.path.join(src_folder, file)
        dst_path = os.path.join(dst_folder, file)
        shutil.move(src_path, dst_path)

    print(f"✅ Spostati {n_to_move} file da {src_folder} a {dst_folder}")

# Sposta il 20%
move_percentage_of_files(trainA_folder, testA_folder, 0.2)
#move_percentage_of_files(trainB_folder, testB_folder, 0.2)

print("✅ Operazione completata!")
